### Figured that Value iteration is model-based approach (need a full knowledge of transition model). You cannot use for the model-free batch learning... (11/01/2023)

In [ ]:
using Printf
using DataFrames
using CSV
using LinearAlgebra

In [ ]:
infile = "data/small.csv"
df = CSV.File(infile) |> DataFrame
# data_mat = Matrix(df);

x = [mod(df.s[j], 10)!=0 ? mod(df.s[j], 10) : 10 for j in 1:size(df,1)] 
y = [mod(df.s[j], 10)!=0 ? df.s[j] ÷ 10 + 1 : df.s[j] ÷ 10  for j in 1:size(df,1)] 

df = insertcols!(df, 2, :s_i => [x[i] for i in 1:size(df,1)])
df = insertcols!(df, 3, :s_j => [y[j] for j in 1:size(df,1)])


In [ ]:
struct AsynchValueIteration
    k_max 
end 

struct MDP 
    γ 
    S
    A 
    T 
    R 
    TR 
end 

In [ ]:
function lookahead(P::MDP, U, s, a)
    S,T,R,γ = P.S, P.T, P.R, P.γ
    return R(s,a) + γ * sum(T(s,a,s_) for s_ in S)
end

# Bellman backup 
function backup(P::MDP, U, s)
    return maximum(lookahead(P,U,s,a) for a in P.A)
end 

In [ ]:
struct ValueFunctionPolicy
    P  # problem
    U  # utility function 
end

function greedy(P::MDP, U, s)
    u, a = findmax(a->lookahead(P,U,s,a), P.A)
    return (a=a, u=u)
end

(π::ValueFunctionPolicy)(s) = greedy(π.P, π.U, s).a

In [ ]:
function solve(M::AsynchValueIteration, P::MDP)
    U = [0.0 for s in P.S]
    for k = 1:M.k_max
        for (i,s) in enumerate(P.S)
            U[i] = backup(P, U, s)
        end 
    end
    return ValueFunctionPolicy(P,U)
end

In [3]:
S = [[[x,y] for x in 1:10] for y in 1:10] 
A = [1,2,3,4]


10-element Vector{Vector{Vector{Int64}}}:
 [[1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [6, 1], [7, 1], [8, 1], [9, 1], [10, 1]]
 [[1, 2], [2, 2], [3, 2], [4, 2], [5, 2], [6, 2], [7, 2], [8, 2], [9, 2], [10, 2]]
 [[1, 3], [2, 3], [3, 3], [4, 3], [5, 3], [6, 3], [7, 3], [8, 3], [9, 3], [10, 3]]
 [[1, 4], [2, 4], [3, 4], [4, 4], [5, 4], [6, 4], [7, 4], [8, 4], [9, 4], [10, 4]]
 [[1, 5], [2, 5], [3, 5], [4, 5], [5, 5], [6, 5], [7, 5], [8, 5], [9, 5], [10, 5]]
 [[1, 6], [2, 6], [3, 6], [4, 6], [5, 6], [6, 6], [7, 6], [8, 6], [9, 6], [10, 6]]
 [[1, 7], [2, 7], [3, 7], [4, 7], [5, 7], [6, 7], [7, 7], [8, 7], [9, 7], [10, 7]]
 [[1, 8], [2, 8], [3, 8], [4, 8], [5, 8], [6, 8], [7, 8], [8, 8], [9, 8], [10, 8]]
 [[1, 9], [2, 9], [3, 9], [4, 9], [5, 9], [6, 9], [7, 9], [8, 9], [9, 9], [10, 9]]
 [[1, 10], [2, 10], [3, 10], [4, 10], [5, 10], [6, 10], [7, 10], [8, 10], [9, 10], [10, 10]]

In [ ]:
rl   = AsynchValueIteration(100)
prob = MDP()